In [ ]:
# put this in it's own codeblock because it'll keep 
# running otherwise and eventually move to the root dir lol
%cd ../..

In [ ]:
from lib.service.database import DatabaseService
from lib.defaults import INSTANCE_CFG
import os

os.makedirs('_out_img', exist_ok=True)

db = DatabaseService.create(INSTANCE_CFG[3].database, 1)
await db.wait_till_running()

In [ ]:
from dataclasses import dataclass, field
import locale
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.colors import LogNorm, Normalize
from matplotlib import rc
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sqlalchemy import text
import warnings

@dataclass
class Target:
    name: str
    figsize: tuple[int, int]
    leg_loc: str
    src_loc: str
    split: str
    ntile: int
    on_unzoned: bool = field(default=True)
    intersection: bool | None = field(default=None)

    @property
    def cmap_linspace_ticks(self):
        return np.linspace(0, 1, self.ntile).tolist()

    @property
    def cardinal_cbar_position(self):
        return 'left' if self.split == 'v' else 'right'

targets = [
    Target('Ku-ring-gai', (22, 20), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Randwick', (12, 20), leg_loc="upper right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Camden', (25, 25), leg_loc="upper right", src_loc="lower left", split='v', ntile=10),
    Target('Blacktown', (25, 25), leg_loc="upper right", src_loc="lower left", split='v', ntile=10),
    Target('Campbelltown (NSW)', (25, 25), leg_loc="upper right", src_loc="lower left", split='v', ntile=10),
    Target('Lane Cove', (25, 25), leg_loc="upper right", src_loc="lower left", split='v', ntile=10),
    Target('Cumberland', (25, 15), leg_loc="upper right", src_loc="lower left", split='h', ntile=10),
    Target('Sydney', (15, 20), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Woollahra', (20, 18), leg_loc="lower right", src_loc="lower left", split='v', ntile=10),
    Target('Waverley', (20, 18), leg_loc="lower right", src_loc="lower left", split='v', ntile=10),
    Target('Burwood', (15, 20), leg_loc="lower right", src_loc="lower left", split='v', ntile=10),
    Target('Strathfield', (15, 20), leg_loc="lower right", src_loc="lower left", split='v', ntile=10),
    Target('The Hills', (20, 50), leg_loc="upper right", src_loc="lower left", split='v', ntile=10),
    Target('Parramatta', (22, 20), leg_loc="upper left", src_loc="lower left", split='v', ntile=10),
    Target('Inner West', (22, 20), leg_loc="upper left", src_loc="lower left", split='v', ntile=10),
    Target('Canada Bay', (25, 20), leg_loc="upper left", src_loc="lower left", split='h', ntile=10),
    Target('Fairfield', (25, 15), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='h', ntile=10),
    Target('Liverpool', (35, 20), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='h', ntile=10),
    Target('Canterbury-Bankstown', (22, 20), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Georges River', (22, 20), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Bayside (NSW)', (22, 20), leg_loc="lower right", src_loc="lower left", split='v', ntile=10),
    Target('Hunters Hill', (22, 20), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Mosman', (22, 20), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('North Sydney', (22, 20), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Northern Beaches', (20, 30), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Ryde', (22, 20), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Sutherland', (22, 20), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Willoughby', (22, 15), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Hornsby', (22, 30), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
    Target('Penrith', (32, 30), leg_loc="lower right", src_loc="lower left", on_unzoned=False, split='v', ntile=10),
]

rc('text', usetex=False)

ranked_zones = ['B1','B2','B3','B4','B5','B6','B7','E1','E2','E3','E4','E5',
                'IN1','IN2','MU1','R1','R2','R3','R4','R5','RU1','RU2','RU3','RU4','RU5','RU6',
                'SP1','SP3','SP4','SP5','W1','W3','W2','W4', 'C3', 'C4']
hidden_zones = ['C1', 'C2', 'SP2','RE1','RE2']

async def execute(query):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        async with db.async_connect() as conn:
            await conn.execute(query);
            await conn.commit();

def create_source_annonation():
    source_box = AnchoredText(
        "Created by Angus Thomsen (@Angus_KST)\n"\
        "Source: NSW Valuer General, published November 2024 accessed 2025\n"\
        "Shapes: ABS Australian Bureau of Statistics & NSW Spatial",
        loc=target.src_loc,
        prop=dict(size=9),
        frameon=True,  # Draw a box around the text
    )
    source_box.patch.set_boxstyle("round,pad=0")
    source_box.patch.set_edgecolor("white")
    source_box.patch.set_facecolor("white")
    return source_box

def format_aud(value):
    # Temporarily set locale
    current_locale = locale.getlocale(locale.LC_ALL)
    try:
        # Set the desired locale
        locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
        # Format the currency
        return locale.currency(value, grouping=True)
    finally:
        # Restore the original locale
        locale.setlocale(locale.LC_ALL, current_locale)

RANK_COLORS, CARDINAL_COLORS = 'viridis', 'magma'
NTILE_KIND = { 5: 'Quintile', 8: 'Octile', 10: 'Dectile' }

for target in targets:
    await execute(f"""
        DROP TABLE IF EXISTS lga_sqm_land_values;

        WITH data AS (
          SELECT p.property_id, p.geometry, (lv.land_value / ST_Area(p.geometry::geography)) AS lv
          FROM abs.lga_2024 l
          JOIN nsw_lrs.property_geometry p ON ST_Within(p.geometry, l.geometry)
          JOIN LATERAL (
            SELECT z.zone_code FROM nsw_lrs.zone_observation z WHERE z.property_id = p.property_id
            ORDER BY z.effective_date DESC LIMIT 1) z ON TRUE
          JOIN LATERAL (
            SELECT v.land_value FROM nsw_vg.land_valuation v
            WHERE v.property_id = p.property_id AND v.land_value IS NOT NULL
            ORDER BY v.effective_date DESC LIMIT 1) lv ON TRUE
          WHERE l.lga_name ILIKE '{target.name}'
            AND z.zone_code IS NOT NULL
            AND NOT (z.zone_code IN ('{"', '".join(hidden_zones)}')))
        SELECT property_id, geometry as g, lv, 
               NTILE({target.ntile}) OVER (ORDER BY lv) AS percentile
          INTO lga_sqm_land_values FROM data;
    """)

    lga_query = f"""
      SELECT geometry as g FROM abs.lga_2024 lga WHERE lga_name ILIKE '{target.name}'
    """
            
    cardinal_query = "SELECT * FROM lga_sqm_land_values"
    
    bins_df = pd.read_sql(f"""
        SELECT percentile, FLOOR(MIN(lv)) AS lower, FLOOR(MAX(lv)) AS upper
          FROM lga_sqm_land_values
          GROUP BY percentile
          ORDER BY percentile;
    """, db.engine())
    
    bins_df['lower'] = bins_df['lower'].apply(format_aud)
    bins_df['upper'] = bins_df['upper'].apply(format_aud)
    cardinal_ticks = [
        f"≤ {bins_df.loc[0, "upper"]}",
        *[f"≤ {bins_df.loc[i, 'lower']}" for i in range(1, len(bins_df) - 1)],
        f"≥ {bins_df.loc[len(bins_df)-1, "lower"]}",
    ]    

    fig, ax = plt.subplots(1, 1, figsize=target.figsize)

    lga_df = gpd.read_postgis(lga_query, db.engine(), geom_col='g')
    lga_df.plot(ax=ax, color='#cccccc', edgecolor="#cccccc")
    
    cardinal_df = gpd.read_postgis("SELECT * FROM lga_sqm_land_values", db.engine(), geom_col='g')
    cardinal_df.plot(ax=ax, column='percentile', cmap=CARDINAL_COLORS)
    
    cardinal_sm = plt.cm.ScalarMappable(cmap=plt.get_cmap(CARDINAL_COLORS), norm=Normalize(vmin=0, vmax=1))
    cardinal_sm.set_array([])
    
    cardinal_cbar = plt.colorbar(cardinal_sm, ax=ax, shrink=0.8, aspect=50, ticks=target.cmap_linspace_ticks, location=target.cardinal_cbar_position)
    cardinal_cbar.set_label("SQM Land Value in AUD", labelpad=10, fontsize=15)
    cardinal_cbar.ax.tick_params(labelsize=12, rotation=67.5) 
    cardinal_cbar.ax.set_yticklabels(cardinal_ticks)
    for label in cardinal_cbar.ax.get_yticklabels():
        label.set_verticalalignment('center')

    custom_legend = [Patch(facecolor='#cccccc', edgecolor="black", label=target.name)]
    ax.legend(handles=custom_legend, loc=target.leg_loc, title="Legend", fontsize=12, title_fontsize=10)
    plt.title("Shared title")
    
    ax.set_title("\nSQM Land Value\nGrouped by %s\nWithin '%s' LGA\n" % (NTILE_KIND[target.ntile], target.name), fontsize=20)
    ax.add_artist(create_source_annonation())
    fig.savefig(f"_out_img/lga_ntile_lvsqm-{target.name.replace(' ', '_')}.png")
    plt.show()
    await execute("""
        DROP TABLE IF EXISTS lga_sqm_land_values;
    """)
    
    